In [1]:
from libs.connector.postgres_connector import PostgresToJsonConnector
from libs.handler.postgres_handler import PostgresHandler
from libs.handler.json_handler import JsonHandler
from libs.log_handler.log_handler import LogHandler

from libs.dataframes.to_types import to_list_of_dicts

import pandas as pd

from libs.dedupe.algo.exact_identifier import exact_identifier
from libs.dedupe.algo.exact_lat_long import exact_lat_long
from libs.dedupe.algo.exact_name_url import exact_name_url


log_handler = PostgresHandler(
    host = 'silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com',
    port = '25060',
    db = 'defaultdb',
    password = 'AVNS_2Lh_hY8r7RVKSfLwbJM',
    username = 'doadmin',
)

logger = LogHandler(db_handler=log_handler)
print (logger)

Default Destination: 1
DB handler host: silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com
DB handler db: defaultdb
DB handler username: doadmin
DB handler schema: public
Connection Alive? Yes


In [2]:



input_qry = '''
    SELECT
        t1.id as location_id,
        t1.latitude,
        t1.longitude,
        t2.id,
        t2.name,
        t2.alternate_name,
        t2.description
    FROM location t1
    LEFT JOIN organization t2
    ON t1.organization_id = t2.id
    '''

# input_qry = "select t1.identifier, t2.address_1, t2.address_2, t2.city, t2.state_province, t2.postal_code, t3.latitude, t3.longitude, t4.id, t4.website, t4.name from organization t4 left join location t3 on t4.id = t3.organization_id left join address t2 on t3.id = t2.location_id left join organization_identifier t1 on t1.organization_id = t4.id"

input_handler = PostgresHandler(
    host = 'silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com',
    port = '25060',
    db = 'defaultdb',
    password = 'AVNS_2Lh_hY8r7RVKSfLwbJM',
    username = 'doadmin',
    query = input_qry,
    schema = 'connect211_wa211_hsds3',
    env_prefix = 'POSTGRES'
)

output_handler = JsonHandler()

print (input_handler)
print (output_handler)

Host: silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com
Port: 25060
Database: defaultdb
Username: doadmin
Schema: connect211_wa211_hsds3
Connection Alive? Yes


In [3]:
latlong_conn = PostgresToJsonConnector(
    input_handler=input_handler,
    output_handler=output_handler,
    log_handler=logger,
)

In [4]:
latlong_conn.df


,location_id,latitude,longitude,id,name,alternate_name,description
0,19d3ef922034d505eced42c68cd45bc6,47.977084,-122.206165,d15710b284043e2b9c6e5bbd6fdf0606,Snohomish County Human Services Department,None,Provides funding to contract and oversee human...
1,b43cfd08ed1254f071fc4bbcb2c9d6a9,46.297613,-118.320413,c499eb2b1dcd64dd4ab22fa8d7b7160d,Prescott School District,None,Provides educational services to children who ...
2,45c32b99b59ad149e16162bff09c5ec5,47.474372,-122.274069,4a9442e89a6cc759c8370033e66b9813,Tukwila School District No. 406,None,"Provides basic education, vocational education..."
3,3792f3af1f45798c0735b50c084154fd,47.159253,-122.519972,5758f3424987a7beffa3996f6f8ba536,Clover Park School District # 121,None,Offers a public school education to Lakewood a...
4,bebcf57ac1f6a633daa6bf13b11f9007,48.533469,-117.889710,c508ae1f6068c8d6b54d3df50eddbe01,Providence Northeast Washington Medical Group,None,Provides physician medical services to residen...
...,...,...,...,...,...,...,...
9553,27000165ee1018671a864a3d3fc94993,46.798439,-122.868878,13c7b5421ca9d52e2d7e42bf9ee4825a,Tenino Food Bank Plus,None,Emergency food to any residents of Tenino or B...
9554,c4164a273fb3fafab30fc2bf19a79c87,46.674723,-122.969784,8e45a70d477a66ca71de6d5ed8deaff3,Catholic Community Services - Senior Programs,None,Provides rides for eligible seniors to medical...
9555,b5a961a7ce3f78283f3c1e970f2598d9,47.202016,-121.991691,9a7a3118920b5a0c83538975f4f32c0a,Enumclaw Kiwanis Food Bank,None,Operates a food pantry and government commodit...
9556,37d1ac37899531842cf1bbc82f173800,0.000000,0.000000,c892188d961d63f4d453f3e5db9b9fc4,Tabernacle Missionary Baptist Church,None,Provides Transitional housing to single female...


In [5]:
job_id = latlong_conn.transform(exact_lat_long)
print (job_id)


236dd79e-853f-4c28-b085-02cd0cb8ea5f


In [6]:
print (latlong_conn.df)

                           location_id   latitude   longitude  \
0     19d3ef922034d505eced42c68cd45bc6  47.977084 -122.206165   
1     b43cfd08ed1254f071fc4bbcb2c9d6a9  46.297613 -118.320413   
2     45c32b99b59ad149e16162bff09c5ec5  47.474372 -122.274069   
3     3792f3af1f45798c0735b50c084154fd  47.159253 -122.519972   
4     bebcf57ac1f6a633daa6bf13b11f9007  48.533469  -117.88971   
...                                ...        ...         ...   
9300  27000165ee1018671a864a3d3fc94993  46.798439 -122.868878   
9301  c4164a273fb3fafab30fc2bf19a79c87  46.674723 -122.969784   
9302  b5a961a7ce3f78283f3c1e970f2598d9  47.202016 -121.991691   
9303  37d1ac37899531842cf1bbc82f173800        0.0         0.0   
9304  91cebb5198e046efa2037c77db4174d2  46.976064 -123.886306   

                                    id  \
0     d15710b284043e2b9c6e5bbd6fdf0606   
1     c499eb2b1dcd64dd4ab22fa8d7b7160d   
2     4a9442e89a6cc759c8370033e66b9813   
3     5758f3424987a7beffa3996f6f8ba536   
4     c50

In [7]:
logger.get(job_id)

[RealDictRow([('log_message',
               {'results': [{'location_id': '19d3ef922034d505eced42c68cd45bc6',
                  'latitude': 47.9770837,
                  'longitude': -122.2061653,
                  'id': 'd15710b284043e2b9c6e5bbd6fdf0606',
                  'name': 'Snohomish County Human Services Department',
                  'alternate_name': '',
                  'description': 'Provides funding to contract and oversee human services.'},
                 {'location_id': 'b43cfd08ed1254f071fc4bbcb2c9d6a9',
                  'latitude': 46.297613,
                  'longitude': -118.3204129,
                  'id': 'c499eb2b1dcd64dd4ab22fa8d7b7160d',
                  'name': 'Prescott School District',
                  'alternate_name': '',
                  'description': 'Provides educational services to children who lice within the school district boundaries.'},
                 {'location_id': '45c32b99b59ad149e16162bff09c5ec5',
                  'latitude': 47

In [8]:

input_qry = '''
    SELECT
        t1.id,
        t1.organization_id,
        t2.name,
        t1.identifier
    FROM organization_identifier t1
    LEFT JOIN organization t2
    ON t1.organization_id = t2.id
    '''

input_handler = PostgresHandler(
    host = 'silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com',
    port = '25060',
    db = 'defaultdb',
    password = 'AVNS_2Lh_hY8r7RVKSfLwbJM',
    username = 'doadmin',
    query = input_qry,
    schema = 'connect211_wa211_hsds3',
    env_prefix = 'POSTGRES'
)

identifier_conn = PostgresToJsonConnector(
    input_handler=input_handler,
    output_handler=output_handler,
    log_handler=logger,
)
print (identifier_conn.input_handler)

Host: silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com
Port: 25060
Database: defaultdb
Username: doadmin
Schema: connect211_wa211_hsds3
Connection Alive? Yes


In [9]:
print (identifier_conn.df)

                                    id                   organization_id  \
0     ef9eec0b9a2cb13f7931c195eb1fbc43  d15710b284043e2b9c6e5bbd6fdf0606   
1     7028e95b07fc0591aa0e7ee9ca7a3573  c499eb2b1dcd64dd4ab22fa8d7b7160d   
2     34a16595f48d871e2ff12f0a08e671df  4a9442e89a6cc759c8370033e66b9813   
3     1deb82bb7a57199dc151839e7884f46f  5758f3424987a7beffa3996f6f8ba536   
4     2958de81055245813cf8d8d11eba310d  c508ae1f6068c8d6b54d3df50eddbe01   
...                                ...                               ...   
4376  1718ac23913e1c03c80898ac6b37b50c  13c7b5421ca9d52e2d7e42bf9ee4825a   
4377  2c589bf22bbb1ac93d69fa75e7fd0583  8e45a70d477a66ca71de6d5ed8deaff3   
4378  d19a493678eaed73fe9c217a282a9e1c  9a7a3118920b5a0c83538975f4f32c0a   
4379  b55322af6d2b16023f25a8213ad7ffd8  c892188d961d63f4d453f3e5db9b9fc4   
4380  37216f417bee0f86694bf1899e573aa8  2be6ecaf096c23e9de3b8e304ae5256d   

                                               name  identifier  
0        Snohomish Co

In [10]:
job_id = identifier_conn.transform(exact_identifier)
print (job_id)


ea89440c-7a3d-4e25-871b-6c604d061313


In [11]:
print (identifier_conn.df)

                                    id                   organization_id  \
0     ef9eec0b9a2cb13f7931c195eb1fbc43  d15710b284043e2b9c6e5bbd6fdf0606   
1     7028e95b07fc0591aa0e7ee9ca7a3573  c499eb2b1dcd64dd4ab22fa8d7b7160d   
2     34a16595f48d871e2ff12f0a08e671df  4a9442e89a6cc759c8370033e66b9813   
3     1deb82bb7a57199dc151839e7884f46f  5758f3424987a7beffa3996f6f8ba536   
4     2958de81055245813cf8d8d11eba310d  c508ae1f6068c8d6b54d3df50eddbe01   
...                                ...                               ...   
4276  bca7f0f3e1109b3b73d3ce08adfa9704  2685b9356671b30b1da9cf79179a4396   
4277  1718ac23913e1c03c80898ac6b37b50c  13c7b5421ca9d52e2d7e42bf9ee4825a   
4278  d19a493678eaed73fe9c217a282a9e1c  9a7a3118920b5a0c83538975f4f32c0a   
4279  b55322af6d2b16023f25a8213ad7ffd8  c892188d961d63f4d453f3e5db9b9fc4   
4280  37216f417bee0f86694bf1899e573aa8  2be6ecaf096c23e9de3b8e304ae5256d   

                                               name  identifier  
0        Snohomish Co

In [12]:
logger.get(job_id)

[RealDictRow([('log_message',
               {'results': [{'id': 'ef9eec0b9a2cb13f7931c195eb1fbc43',
                  'organization_id': 'd15710b284043e2b9c6e5bbd6fdf0606',
                  'name': 'Snohomish County Human Services Department',
                  'identifier': ''},
                 {'id': '7028e95b07fc0591aa0e7ee9ca7a3573',
                  'organization_id': 'c499eb2b1dcd64dd4ab22fa8d7b7160d',
                  'name': 'Prescott School District',
                  'identifier': ''},
                 {'id': '34a16595f48d871e2ff12f0a08e671df',
                  'organization_id': '4a9442e89a6cc759c8370033e66b9813',
                  'name': 'Tukwila School District No. 406',
                  'identifier': ''},
                 {'id': '1deb82bb7a57199dc151839e7884f46f',
                  'organization_id': '5758f3424987a7beffa3996f6f8ba536',
                  'name': 'Clover Park School District # 121',
                  'identifier': ''},
                 {'id': '2958

In [13]:
input_qry = '''
    SELECT
        id,
        name,
        website
    FROM organization
    '''


input_handler = PostgresHandler(
    host = 'silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com',
    port = '25060',
    db = 'defaultdb',
    password = 'AVNS_2Lh_hY8r7RVKSfLwbJM',
    username = 'doadmin',
    query = input_qry,
    schema = 'connect211_wa211_hsds3',
    env_prefix = 'POSTGRES'
)

url_conn = PostgresToJsonConnector(
    input_handler=input_handler,
    output_handler=output_handler,
    log_handler=logger,
)
print (url_conn.input_handler)

Host: silobuster-dev-db-do-user-12298230-0.b.db.ondigitalocean.com
Port: 25060
Database: defaultdb
Username: doadmin
Schema: connect211_wa211_hsds3
Connection Alive? Yes


In [14]:
print (url_conn.df)

                                    id  \
0     d15710b284043e2b9c6e5bbd6fdf0606   
1     c499eb2b1dcd64dd4ab22fa8d7b7160d   
2     4a9442e89a6cc759c8370033e66b9813   
3     5758f3424987a7beffa3996f6f8ba536   
4     c508ae1f6068c8d6b54d3df50eddbe01   
...                                ...   
4376  13c7b5421ca9d52e2d7e42bf9ee4825a   
4377  8e45a70d477a66ca71de6d5ed8deaff3   
4378  9a7a3118920b5a0c83538975f4f32c0a   
4379  c892188d961d63f4d453f3e5db9b9fc4   
4380  2be6ecaf096c23e9de3b8e304ae5256d   

                                               name  \
0        Snohomish County Human Services Department   
1                          Prescott School District   
2                   Tukwila School District No. 406   
3                 Clover Park School District # 121   
4     Providence Northeast Washington Medical Group   
...                                             ...   
4376                          Tenino Food Bank Plus   
4377  Catholic Community Services - Senior Programs   


In [15]:
job_id = url_conn.transform(exact_name_url)
print (job_id)

cb1ad95f-d189-476e-a33a-869787b4213a


In [16]:
print (url_conn.df)

                                    id  \
0     d15710b284043e2b9c6e5bbd6fdf0606   
1     c499eb2b1dcd64dd4ab22fa8d7b7160d   
2     4a9442e89a6cc759c8370033e66b9813   
3     5758f3424987a7beffa3996f6f8ba536   
4     c508ae1f6068c8d6b54d3df50eddbe01   
...                                ...   
4368  13c7b5421ca9d52e2d7e42bf9ee4825a   
4369  8e45a70d477a66ca71de6d5ed8deaff3   
4370  9a7a3118920b5a0c83538975f4f32c0a   
4371  c892188d961d63f4d453f3e5db9b9fc4   
4372  2be6ecaf096c23e9de3b8e304ae5256d   

                                               name  \
0        Snohomish County Human Services Department   
1                          Prescott School District   
2                   Tukwila School District No. 406   
3                 Clover Park School District # 121   
4     Providence Northeast Washington Medical Group   
...                                             ...   
4368                          Tenino Food Bank Plus   
4369  Catholic Community Services - Senior Programs   


In [17]:
logger.get(job_id)

[RealDictRow([('log_message',
               {'results': [{'id': 'd15710b284043e2b9c6e5bbd6fdf0606',
                  'name': 'Snohomish County Human Services Department',
                  'website': 'https://snohomishcountywa.gov/191/Human-Services'},
                 {'id': 'c499eb2b1dcd64dd4ab22fa8d7b7160d',
                  'name': 'Prescott School District',
                  'website': 'www.prescott.k12.wa.us'},
                 {'id': '4a9442e89a6cc759c8370033e66b9813',
                  'name': 'Tukwila School District No. 406',
                  'website': 'http://tukwilaschools.org'},
                 {'id': '5758f3424987a7beffa3996f6f8ba536',
                  'name': 'Clover Park School District # 121',
                  'website': 'http://www.cloverpark.k12.wa.us'},
                 {'id': 'c508ae1f6068c8d6b54d3df50eddbe01',
                  'name': 'Providence Northeast Washington Medical Group',
                  'website': 'http://newmg.org/'},
                 {'id